In [125]:
import urllib2
from BeautifulSoup import BeautifulSoup
import time
import pandas as pd

In [44]:

chili_url = 'http://allrecipes.com/recipes/173/soups-stews-and-chili/chili/'


In [45]:
chili_links = []
for page in range(1,24):
    #there are 23 pages of chili urls
    chili_page = chili_url + "?page=" + str(page)
    response = urllib2.urlopen(chili_page)
    soup = BeautifulSoup(response.read())
    
    for link in soup.findAll('a'):
        content = str(link.get('href'))
        if ("chili"  in content) and (('recipe/') in content):
            #these are individual recipes
            chili_links.append(content)
        
    


In [51]:
#I managed to get quite a few duplicates, but such is web scraping.  Let's dedupe my list

print len(set(chili_links))

chili_set = set(chili_links)

430


In [122]:
def recipe_from_url(url):
    try:
        response = urllib2.urlopen(url)
        soup = BeautifulSoup(response.read())
        #meta information
        title = 'Unknown'
        rating = 'Unknown'
        reviews = 'Unknown'
        description = 'Unknown'
        cals = 'Unknown'
        fat = 'Unknown'
        carbs = 'Unknown'
        protein = 'Unknown'
        
        title = soup.title.text.encode('utf-8')
        rating = soup.find('meta', attrs={'itemprop':'ratingValue'})['content'].encode('utf-8')
        reviews = soup.find('span', attrs={'class':'review-count'}).text.encode('utf-8')
        description = soup.find('div', attrs={'class':'submitter__description'}).text.encode('utf-8')
        cals = soup.find('li', attrs={'itemprop':'calories'}).text.encode('utf-8')
        fat = soup.find('li', attrs={'itemprop':'fatContent'}).text.encode('utf-8')
        carbs = soup.find('li', attrs={'itemprop':'carbohydrateContent'}).text.encode('utf-8')
        protein = soup.find('li', attrs={'itemprop':'proteinContent'}).text.encode('utf-8')
        #ingredients
        ingredient_list = []
        all_ingredients = soup.findAll('span', attrs={'itemprop':'ingredients'})
        for i in all_ingredients:
            ingredient_list.append(i.text.encode('utf-8'))
            
        return {'title':title, 'rating':rating, 'reviews':reviews, 'description':description, 'calories':cals,
            'fat':fat, 'carbs':carbs, 'protein':protein, 'ingredients':ingredient_list}   
    except:
        print "ERROR ON %s" % url
        return "PARSE_ERROR"
    

In [123]:
base_url = 'http://allrecipes.com'
chili_recipe_list = []

for path in chili_set:
    chili_url = base_url + path
    result = recipe_from_url(chili_url)
    if result != "PARSE_ERROR":
        chili_recipe_list.append(result)
    time.sleep(3) # don't pound the OKCupid servers
    

ERROR ON http://allrecipes.com/recipe/151205/jimmy-dean-hearty-holiday-chili/
ERROR ON http://allrecipes.com/recipe/215228/simply-sensational-chili/
ERROR ON http://allrecipes.com/recipe/222977/quick-and-healthy-turkey-chili/
ERROR ON http://allrecipes.com/recipe/151628/2-bean-chili/


In [124]:
len(chili_recipe_list)

426

In [126]:
chili_df = pd.DataFrame(chili_recipe_list)

In [129]:
chili_df.to_json("chili.json")

In [130]:
chili_df.to_csv("chili_recipes.csv", na_rep="NA")